<a href="https://colab.research.google.com/github/Abhi-GX/Wealthwise/blob/main/agent34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install yfinance flask pyngrok
# !pip install serpapi
!pip install dotenv

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from serpapi import Client
from typing import Dict, List, Union
import torch
import numpy as np
import os
# from dotenv import load_dotenv

class NewsSentimentAnalyzer:
    def __init__(self):
        """
        Initialize the analyzer with SerpAPI and FinBERT models
        """
        # Load environment variables
        # load_dotenv()

        # Setup SerpAPI
        self.serpapi_key = "aa7b9ef6bd5fc7bbd7badf1daa212bfcbef27f932cdf396a19cef92bcbf88c95"
        if not self.serpapi_key:
            raise ValueError("SERPAPI_KEY must be set in environment variables")

        # Setup FinBERT
        self.finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
        self.finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

    def analyze_company_sentiment(
        self,
        company_name: str,
        num_articles: int = 10
    ) -> Dict[str, Union[float, List[Dict], str]]:
        """
        Fetch news articles about a company and analyze their sentiment in one go

        Args:
            company_name (str): Name of the company to analyze
            num_articles (int): Number of articles to retrieve and analyze

        Returns:
            dict: Analysis results including reputation score, sentiment distribution,
                 and detailed article analysis
        """
        try:
            # Fetch news articles
            client = Client(api_key=self.serpapi_key)
            results = client.search({
                'engine': 'google_news',
                'q': company_name,
                'gl': 'in',
                'hl': 'en'
            })

            news_articles = results.get('news_results', [])[:num_articles]

            if not news_articles:
                return {
                    "status": "error",
                    "message": f"No news articles found for {company_name}"
                }

            # Analyze sentiment for each article
            article_analyses = []
            sentiment_counts = {'Positive': 0, 'Neutral': 0, 'Negative': 0}

            for article in news_articles:
                # Analyze sentiment
                inputs = self.finbert_tokenizer(
                    article['title'],
                    return_tensors="pt",
                    truncation=True,
                    max_length=512
                )

                with torch.no_grad():
                    outputs = self.finbert_model(**inputs)
                    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

                sentiment_scores = predictions.numpy()[0]
                label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
                sentiment = label_map[np.argmax(sentiment_scores)]

                # Update counts
                sentiment_counts[sentiment] += 1

                # Store analysis
                article_analyses.append({
                    'title': article['title'],
                    'source': article.get('source', {}).get('name', 'Unknown'),
                    'link': article.get('link', ''),
                    'sentiment': sentiment,
                    'sentiment_scores': {
                        'Negative': float(sentiment_scores[0]),
                        'Neutral': float(sentiment_scores[1]),
                        'Positive': float(sentiment_scores[2])
                    }
                })

            # Calculate metrics
            total_articles = len(news_articles)
            sentiment_distribution = {
                sentiment: (count/total_articles * 100)
                for sentiment, count in sentiment_counts.items()
            }

            # Calculate reputation score (0-100 scale)
            sentiment_weights = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
            reputation_score = sum([
                sentiment_weights[analysis['sentiment']]
                for analysis in article_analyses
            ]) / total_articles * 50 + 50

            return {
                "status": "success",
                "company_name": company_name,
                "reputation_score": round(reputation_score, 2),
                "sentiment_distribution": {
                    k: round(v, 2) for k, v in sentiment_distribution.items()
                },
                "analyzed_articles": article_analyses,
                "total_articles": total_articles
            }

        except Exception as e:
            return {
                "status": "error",
                "message": f"Analysis failed: {str(e)}"
            }

In [ ]:
analyzer = NewsSentimentAnalyzer()

In [ ]:
class CompanyNewsSentimentTool(BaseTool):
    """
    Company News Sentiment Analysis Tool for comprehensive reputation analysis.

    This tool analyzes news articles and sentiment for a specific company.

    Args:
        company_name (str, required): The name of the company to analyze.
            Example: 'Apple Inc.', 'Google'
        num_articles (int, optional): Number of articles to analyze. Default is 10.

    Returns:
        str: Formatted analysis results including reputation score and sentiment distribution.

    Usage Example:
        tool = CompanyNewsSentimentTool()
        analysis = tool.run(company_name='Apple Inc.', num_articles=5)

    Note:
        - Requires valid company name
        - Requires initialized NewsSentimentAnalyzer
        - Returns error message if analysis fails
    """
    name: str = "Company News Sentiment Analysis Tool"
    description: str = (
        "Analyzes news sentiment and calculates reputation score for a given company. "
        "Input requires a company name and optional number of articles to analyze."
    )

    def __init__(self, analyzer: NewsSentimentAnalyzer):
        """
        Initialize the tool with a NewsSentimentAnalyzer instance.

        Args:
            analyzer (NewsSentimentAnalyzer): Pre-initialized analyzer instance
        """
        super().__init__()
        self.analyzer = analyzer

    def _run(self, company_name: str, num_articles: int = 10) -> str:
        """
        Analyze news sentiment for the specified company.

        Args:
            company_name (str): Name of the company to analyze
            num_articles (int): Number of articles to analyze (default: 10)

        Returns:
            str: Formatted analysis results
        """
        try:
            results = self.analyzer.analyze_company_sentiment(company_name, num_articles)

            if results["status"] == "success":
                output = [
                    f"Company Analysis: {company_name}",
                    f"Number of Articles Analyzed: {results['total_articles']}",
                    f"Overall Reputation Score: {results['reputation_score']}",
                    "\nSentiment Distribution:",
                ]
                for sentiment, percentage in results['sentiment_distribution'].items():
                    output.append(f"  {sentiment}: {percentage}%")
                output.append("\nKey Articles and Their Sentiment:")
                for i, article in enumerate(results['analyzed_articles'][:3], 1):
                    output.append(f"{i}. {article['title']}")
                    output.append(f"   Source: {article['source']}")
                    output.append(f"   Sentiment: {article['sentiment']}")
                    output.append("")

                return "\n".join(output)
            else:
                return f"Analysis Error: {results['message']}"

        except Exception as e:
            return f"Tool Execution Error: {str(e)}"

In [ ]:
# Initialize the analyzer
analyzer = NewsSentimentAnalyzer()

# Analyze a company
results = analyzer.analyze_company_sentiment("Apple Inc.", num_articles=10)

# Check results
if results["status"] == "success":
    print(f"Reputation Score: {results['reputation_score']}")
    print("\nSentiment Distribution:")
    for sentiment, percentage in results['sentiment_distribution'].items():
        print(f"{sentiment}: {percentage}%")

    print("\nTop 3 Articles:")
    for article in results['analyzed_articles'][:3]:
        print(f"- {article['title']} ({article['sentiment']})")
else:
    print(f"Error: {results['message']}")

Reputation Score: 10.0

Sentiment Distribution:
Positive: 10.0%
Neutral: 0.0%
Negative: 90.0%

Top 3 Articles:
- Apple mulls smart home doorbell with support for facial recognition: Report (Negative)
- Apple reportedly eyes home security market with face-scanning smart doorbell (Negative)
- How Did Apple Get So Big? (Negative)


In [ ]:
import yfinance as yf

def get_stock_metrics_with_calculations(ticker_symbol):
    try:
        # Fetch stock data
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        financials = stock.financials
        balance_sheet = stock.balance_sheet

        # Helper function to calculate growth rate
        def calculate_growth(current, past):
            if current and past and past != 0:
                return ((current - past) / past) * 100
            return "N/A"

        # Extract revenue and net income for growth calculations
        try:
            revenue = financials.loc['Total Revenue']
            net_income = financials.loc['Net Income']

            sales_growth_3y = calculate_growth(revenue.iloc[0], revenue.iloc[3]) if len(revenue) > 3 else "N/A"
            sales_growth_5y = calculate_growth(revenue.iloc[0], revenue.iloc[5]) if len(revenue) > 5 else "N/A"
            profit_growth_3y = calculate_growth(net_income.iloc[0], net_income.iloc[3]) if len(net_income) > 3 else "N/A"
            profit_growth_5y = calculate_growth(net_income.iloc[0], net_income.iloc[5]) if len(net_income) > 5 else "N/A"
        except Exception as e:
            sales_growth_3y = sales_growth_5y = profit_growth_3y = profit_growth_5y = f"Error: {e}"

        # Extract equity for ROE calculations
        try:
            equity = balance_sheet.loc['Total Stockholder Equity']
            roe_3y = calculate_growth(net_income.iloc[0], equity.iloc[3]) if len(equity) > 3 and len(net_income) > 3 else "N/A"
            roe_5y = calculate_growth(net_income.iloc[0], equity.iloc[5]) if len(equity) > 5 and len(net_income) > 5 else "N/A"
        except Exception as e:
            roe_3y = roe_5y = f"Error: {e}"

        history = stock.history(period="5y")
        try:
            return_1y = calculate_growth(history['Close'].iloc[-1], history['Close'].iloc[-252]) if len(history) > 252 else "N/A"
            return_3y = calculate_growth(history['Close'].iloc[-1], history['Close'].iloc[-756]) if len(history) > 756 else "N/A"
            return_5y = calculate_growth(history['Close'].iloc[-1], history['Close'].iloc[-1260]) if len(history) > 1260 else "N/A"
        except Exception as e:
            return_1y = return_3y = return_5y = f"Error: {e}"
        metrics = {
            "Sales Growth 3 Years": sales_growth_3y,
            "Sales Growth 5 Years": sales_growth_5y,
            "Profit Growth 3 Years": profit_growth_3y,
            "Profit Growth 5 Years": profit_growth_5y,
            "Average ROE 3 Years": roe_3y,
            "Average ROE 5 Years": roe_5y,
            "Return over 1 Year": return_1y,
            "Return over 3 Years": return_3y,
            "Return over 5 Years": return_5y,
            "Price to Free Cash Flow": info.get("priceToFreeCashflow", "N/A"),
            "Price to Earnings (P/E)": info.get("forwardPE", "N/A"),
            "Price to Book (P/B)": info.get("priceToBook", "N/A"),
            "Enterprise Value to EBITDA (EV/EBITDA)": info.get("enterpriseToEbitda", "N/A"),
            "Dividend Yield": info.get("dividendYield", "N/A"),
            "Price to Sales (P/S)": info.get("priceToSalesTrailing12Months", "N/A"),
            "Return on Equity (ROE)": info.get("returnOnEquity", "N/A"),
            "Return on Assets (ROA)": info.get("returnOnAssets", "N/A"),
            "Gross Margin": info.get("grossMargins", "N/A"),
            "Operating Margin": info.get("operatingMargins", "N/A"),
            "Net Profit Margin": info.get("netMargins", "N/A"),
            "Market Capitalization": info.get("marketCap", "N/A"),
            "Enterprise Value": info.get("enterpriseValue", "N/A"),
            "Beta": info.get("beta", "N/A"),
            "Shares Outstanding": info.get("sharesOutstanding", "N/A"),
            "Debt to Equity Ratio": info.get("debtToEquity", "N/A"),
            "Current Ratio": info.get("currentRatio", "N/A"),
            "Quick Ratio": info.get("quickRatio", "N/A"),
            "Free Cash Flow Yield": info.get("freeCashflow", "N/A"),
            "Revenue per Share": info.get("revenuePerShare", "N/A"),
            "EPS Growth": info.get("earningsGrowth", "N/A"),
            "52-Week High": info.get("fiftyTwoWeekHigh", "N/A"),
            "52-Week Low": info.get("fiftyTwoWeekLow", "N/A"),
            "Average Trading Volume": info.get("averageVolume", "N/A"),
            "Relative Strength Index (RSI)": "N/A",  # Requires separate calculation
            "Volatility (Weekly)": "N/A",  # Requires separate calculation
            "Volatility (Monthly)": "N/A",  # Requires separate calculation
        }

        return metrics
    except Exception as e:
        return {"Error": str(e)}

In [ ]:
# Initialize the analyzer
analyzer = NewsSentimentAnalyzer(
    serpapi_key="your_serpapi_key",
    finbert_tokenizer=your_tokenizer,
    finbert_model=your_model
)

# Analyze a company
results = analyzer.analyze_company_sentiment("Apple Inc.", num_articles=10)

# Check results
if results["status"] == "success":
    print(f"Reputation Score: {results['reputation_score']}")
    print("\nSentiment Distribution:")
    for sentiment, percentage in results['sentiment_distribution'].items():
        print(f"{sentiment}: {percentage}%")

    print("\nTop 3 Articles:")
    for article in results['analyzed_articles'][:3]:
        print(f"- {article['title']} ({article['sentiment']})")
else:
    print(f"Error: {results['message']}")

{'Sales Growth 3 Years': 204.4817442464094,
 'Sales Growth 5 Years': 'N/A',
 'Profit Growth 3 Years': -109.25154755205402,
 'Profit Growth 5 Years': 'N/A',
 'Average ROE 3 Years': "Error: 'Total Stockholder Equity'",
 'Average ROE 5 Years': "Error: 'Total Stockholder Equity'",
 'Return over 1 Year': 17.63065304558377,
 'Return over 3 Years': -2.427746212810283,
 'Return over 5 Years': 'N/A',
 'Price to Free Cash Flow': 'N/A',
 'Price to Earnings (P/E)': 7.425688,
 'Price to Book (P/B)': 'N/A',
 'Enterprise Value to EBITDA (EV/EBITDA)': 8.154,
 'Dividend Yield': 'N/A',
 'Price to Sales (P/S)': 0.20689704,
 'Return on Equity (ROE)': 'N/A',
 'Return on Assets (ROA)': 0.02753,
 'Gross Margin': 0.24033001,
 'Operating Margin': 0.049099997,
 'Net Profit Margin': 'N/A',
 'Market Capitalization': 11092370432,
 'Enterprise Value': 41315373056,
 'Beta': 1.399,
 'Shares Outstanding': 657131008,
 'Debt to Equity Ratio': 'N/A',
 'Current Ratio': 0.566,
 'Quick Ratio': 0.422,
 'Free Cash Flow Yield'

In [ ]:
import requests
from crewai_tools import BaseTool
from typing import Dict, Any, Optional

class StockDataTool(BaseTool):
    """
    Stock Data Retrieval Tool for comprehensive financial analysis.

    This tool retrieves detailed financial information for a specific stock.

    Args:
        ticker (str, required): The stock ticker symbol to retrieve data for.
            Example: 'AAPL' for Apple Inc., 'GOOGL' for Alphabet Inc.

    Returns:
        dict: Comprehensive financial data for the specified stock.

    Usage Example:
        tool = StockDataTool()
        stock_data = tool.run(ticker='AAPL')

    Note:
        - Ticker should be a valid stock symbol
        - Requires a working connection to the data source
        - Returns None or raises an error if data cannot be retrieved
    """
    name: str = "Stock Data Retrieval Tool"
    description: str = (
        "Retrieves comprehensive financial data for a given stock ticker. "
        "Input requires a valid stock ticker symbol (e.g., 'AAPL', 'GOOGL')."
    )

    def _run(self, ticker: str):
        """
        Retrieve comprehensive stock data for the specified ticker.

        Args:
            ticker (str): Stock ticker symbol to retrieve data for.

        Returns:
            dict: Comprehensive stock financial information
        """
        return get_comprehensive_stock_data(ticker)

stock_data_tool=StockDataTool()


In [ ]:
import requests
from crewai_tools import BaseTool
from typing import Dict, Any

class SectorPerformanceTool(BaseTool):
    """
    Tool to retrieve current sector performance data.

    Returns current sector performance from Financial Modeling Prep API.
    """
    name: str = "Sector Performance Tool"
    description: str = "Retrieves current sector performance data across different sectors"

    def _run(self, *args, **kwargs) -> Dict[str, Any]:
        """
        Fetch current sectors performance data.

        Returns:
            Dict containing sector performance data or error information
        """
        url = "https://financialmodelingprep.com/api/v3/sectors-performance?apikey=jC73g3RPCMZgUjI53dgjJsyuUszaZ2MX"
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return {"error": f"Request failed: {str(e)}"}

class HistoricalSectorPerformanceTool(BaseTool):
    """
    Tool to retrieve historical sector performance data.

    Allows agent to specify from_date and to_date parameters freely.
    """
    name: str = "Historical Sector Performance Tool"
    description: str = "Retrieves historical sector performance data between specified dates"

    def _run(self, from_date: str, to_date: str, *args, **kwargs) -> Dict[str, Any]:
        """
        Retrieve historical sector performance for specified date range.

        Args:
            from_date (str): Start date for performance calculation
            to_date (str): End date for performance calculation

        Returns:
            Dict containing historical sector performance data
        """
        url = f"https://financialmodelingprep.com/api/v3/historical-sectors-performance?from={from_date}&to={to_date}&apikey=jC73g3RPCMZgUjI53dgjJsyuUszaZ2MX"

        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return {"error": f"Request failed: {str(e)}"}
# Create tool instances with default API key
sector_performance_tool = SectorPerformanceTool()
historical_performance_tool = HistoricalSectorPerformanceTool()


In [ ]:
user_input_agent = Agent(
    role="Financial Intake Specialist",
    goal="Transform subjective investor inputs into a comprehensive, quantifiable investment profile that guides subsequent analytical processes.",
    backstory="A meticulous financial intake specialist who transforms raw investor preferences into structured, actionable investment parameters. Skilled in extracting nuanced investment goals, risk appetite, and financial constraints through precise questioning and advanced data validation techniques.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [ ]:
risk_assessment_agent = Agent(
    role="Comprehensive Risk Strategist",
    goal="Generate nuanced risk profiles that illuminate sector-specific vulnerabilities and investment strategy constraints, providing actionable insights for sector selection.",
    backstory="A meticulous risk architect who deconstructs investment landscapes by mapping intricate risk interconnections across various economic sectors. Specializes in translating complex risk dynamics into strategic sector recommendations that balance potential returns with comprehensive risk mitigation strategies.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

In [ ]:
trading_strategy_agent = Agent(
    role="Market Opportunity Architect",
    goal="Develop a strategic framework that identifies high-potential sectors and stock characteristics by analyzing market trends, trading patterns, and performance indicators.",
    backstory="A sophisticated market strategist who synthesizes complex trading dynamics, leveraging advanced technical analysis and predictive modeling to uncover hidden market opportunities. Transforms raw market data into a strategic blueprint for sector and stock selection.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

In [ ]:
sector_selection_agent = Agent(
    role="Strategic Indian Sector Investment Advisor",
    goal=(
        "Identify and rank the top-performing sectors in the Indian stock market by analyzing insights from risk assessment and trading strategy agents. "
        "Select at least two top sectors, each with a list of at least six high-potential Indian stocks, leveraging tools for sector performance and historical analysis. "
        "Pass this data, along with relevant insights, to the Stock Information Agent for detailed stock filtering."
    ),
    backstory=(
        "An astute market strategist proficient in analyzing sector-level trends in the Indian stock market using both current and historical data. "
        "Specializes in aligning sector choices with user-specific investment strategies and risk profiles while providing a solid foundation for further stock analysis."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[
        sector_performance_tool,
        historical_performance_tool

    ]
)

In [ ]:
stock_information_agent = Agent(
    role="Precision Indian Stock Data Architect",
    goal=(
        "Filter and select the top five Indian stocks from the list provided by the Sector Selection Agent by evaluating strategic financial metrics. "
        "Leverage insights from risk assessment and trading strategy agents to determine the most relevant metrics for stock performance analysis."
    ),
    backstory=(
        "A meticulous financial data analyst specializing in the Indian stock market, adept at extracting meaningful investment insights from raw stock data. "
        "Excels in identifying high-potential Indian stocks tailored to strategic and risk-adjusted investment goals."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[
        stock_data_tool

    ]
)

In [ ]:
recommendation_engine_agent = Agent(
    role="Personalized Stock Recommendation Specialist",
    goal=(
        "Aggregate and present well-structured stock recommendations based on the refined stock list provided by the Stock Information Agent. "
        "Retrieve current stock prices using scraping and search tools, and provide clear explanations for each stock's selection tailored to the user's financial goals."
    ),
    backstory=(
        "A dedicated financial advisor specializing in synthesizing market insights into actionable stock recommendations. "
        "Utilizes advanced tools to ensure each recommendation aligns with the user's risk tolerance, investment strategy, and sector preferences."
    ),
    verbose=True,
    allow_delegation=True,
    tools=[
        scrape_tool,
        search_tool
    ]
)

In [ ]:
user_input_task = Task(
    description=(
        "Meticulously collect and validate detailed user inputs. "
        "Extract comprehensive investor profile including initial capital, risk tolerance, "
        "and trading strategy. Ensure data integrity through advanced validation techniques. "
        "Prepare a structured input profile for subsequent agent analysis."
    ),
    expected_output=(
        "Validated and structured user investment profile with comprehensive input details."
    ),
    agent=user_input_agent,
)

In [ ]:
risk_assessment_task = Task(
    description=(
        "Conduct a comprehensive risk assessment that goes beyond traditional metrics. "
        "Generate a detailed risk landscape highlighting sector-specific vulnerabilities, "
        "emerging market risks, and strategic risk mitigation approaches. "
        "Produce insights that directly inform sector selection and stock filtering strategies."
    ),
    expected_output=(
        "Comprehensive risk assessment report with sector risk rankings, "
        "potential market vulnerabilities, and strategic risk mitigation recommendations."
    ),
    agent=risk_assessment_agent,
)

In [ ]:
trading_strategy_task = Task(
    description=(
        "Develop a sophisticated market opportunity framework that identifies "
        "ideal sector characteristics and stock selection criteria. Analyze "
        "trading patterns, market trends, and performance indicators to "
        "create a strategic blueprint for sector and stock identification."
    ),
    expected_output=(
        "Strategic framework detailing sector selection criteria, "
        "stock performance indicators, and market opportunity analysis."
    ),
    agent=trading_strategy_agent,
)

In [ ]:
sector_selection_task = Task(
    description=(
        "Analyze insights from the Risk Assessment and Trading Strategy agents to rank and recommend at least two top-performing sectors in the Indian stock market. "
        "For each sector, identify at least six high-potential Indian stocks based on comprehensive performance data and historical trends. "
        "Provide a detailed rationale for sector and stock selection, and pass this data along with the insights to the Stock Information Agent."
    ),
    expected_output=(
        "A ranked list of at least two sectors in the Indian stock market with detailed insights and a list of at least six recommended Indian stocks for each sector. "
        "Additional insights for further processing by the Stock Information Agent."
    ),
    agent=sector_selection_agent
)

In [ ]:
stock_information_task = Task(
    description=(
        "Evaluate the stock list and insights provided by the Sector Selection Agent to identify the top five stocks from the Indian stock market "
        "using relevant financial metrics. Use the insights from Risk Assessment and Trading Strategy agents to guide the selection process."
    ),
    expected_output=(
        "A refined list of the top five stocks from the Indian stock market, with a detailed explanation of the selection criteria "
        "and relevance to the user's investment strategy."
    ),
    agent=stock_information_agent
)

In [ ]:
recommendation_engine_task = Task(
    description=(
        "Retrieve the finalized stock list from the Stock Information Agent and provide detailed recommendations for each stock. "
        "For each stock, include the following details: "
        "- **Stock Name**: Name of the stock. "
        "- **Current Stock Price**: Retrieve the latest stock price using the primary data source. "
        "  If unavailable, use search and scrape tools to find the stock price. "
        "- **Explanation**: Justify the stock's selection based on user inputs such as risk tolerance, investment strategy, and sector preferences. "
        "Ensure fallback methods are employed for missing data and present the information in a clear, well-structured format for user review."
    ),
    expected_output=(
        "A detailed and structured list of recommended stocks, including their names, current prices (using fallback tools if needed), and explanations for selection."
    ),
    agent=recommendation_engine_agent
)

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
   agents=[
       user_input_agent,
       risk_assessment_agent,
       trading_strategy_agent,
       sector_selection_agent,
       stock_information_agent,
       recommendation_engine_agent
   ],

   tasks=[
       user_input_task,
       risk_assessment_task,
       trading_strategy_task,
       sector_selection_task,
       stock_information_task,
       recommendation_engine_task
   ],
   manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7),
   process='sequential'
)

In [ ]:
financial_trading_inputs = {
        'initial_capital': '100000',
        'risk_tolerance': 'Medium',
        'trading_strategy_preference': 'Day Trading',
    }

In [ ]:

result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
!pip install flask pyngrok

In [ ]:
from flask import Flask, request
from pyngrok import ngrok
# from flask_cors import CORS
# CORS(app, resources={r"/stockRecommandation": {"origins": "https://wealthwise.vercel.app"}})

app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello, World!"

# POST route to print body content
@app.route('/stockRecommandation', methods=['POST'])
def stock_recommendation():
    try:
        content = request.json
        result = financial_trading_crew.kickoff(inputs=content)
        return {"status": "success", "received": result}, 200
    except Exception as e:
        print(f"Error: {e}")
        return {"status": "error", "message": str(e)}, 500

# Start ngrok tunnel
NGROK_DOMAIN = "keen-marten-tops.ngrok-free.app"
ngrok.set_auth_token("2lGTrPfKaseITcmgdp5ZXnaCrl6_2ABHvnjSSZ9WpLFvV8a2u")

tunnel_config = {
    "addr": PORT,
    "hostname": NGROK_DOMAIN
}
public_url = ngrok.connect(**tunnel_config)
print(f"Public URL: {public_url}")  # Specify port as an integer and protocol as "http"
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

if __name__ == '__main__':
    app.run(port=5000)

Public URL: NgrokTunnel: "https://keen-marten-tops.ngrok-free.app" -> "http://localhost:5000"
 * ngrok tunnel "NgrokTunnel: "https://keen-marten-tops.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:31:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:31:55] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:32:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:33:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:33:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 05:33:22] "GET /favicon.ico HTTP/1.1" 404 -
